# 01 Data Ingestion & Preprocessing
Dieses Notebook lädt Rohdaten, bereinigt sie und bereitet sie für das Feature-Engineering vor.

## 1. Setup & Imports
- Bibliotheken importieren  
- Verzeichnisstrukturen definieren  
- (Optional) openpyxl installieren  

In [118]:
# 1.1 Installation (einmalig pro Kernel)
%pip install openpyxl --quiet

# 1.2 Imports
import os
import csv
import pandas as pd
import numpy as np

# 1.3 Pfade
RAW_DIR       = os.path.join("..", "data", "raw")
PROCESSED_DIR = os.path.join("..", "data", "processed")

# 1.4 Kontrolle: Rohdateien
print("Rohdateien im Verzeichnis:")
for f in sorted(os.listdir(RAW_DIR)):
    print(" -", f)


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Rohdateien im Verzeichnis:
 - US and Canda States.xlsx
 - car_prices.csv
 - states.csv


## 2. Daten einlesen & Parsing-Fix
- `car_prices.csv` laden  
- Spot-Check auf verschobene Zeilen  
- Zeilen mit fehlendem Datum reparieren  
- Datum danach konvertieren  
- `states` für USA+Kanada laden  

In [119]:
# 2.1 car_prices.csv einlesen (Standard-Pandas)
cars = pd.read_csv(
    os.path.join(RAW_DIR, "car_prices.csv"),
    parse_dates=["saledate"],
    infer_datetime_format=True,
    low_memory=False
)

# 2.2 Spot-Check auf bekanntermaßen verschobene Zeilen
problem_idxs = [408161, 417835, 421289, 424161, 427040]
display(cars.loc[problem_idxs, ["trim","body","saledate"]])

# 2.3 Fix: Zeilen mit NaT im saledate nachjustieren
mask_bad = cars["saledate"].isna()
if mask_bad.any():
    # 2.3.1 trim + falsches body zusammenfügen
    cars.loc[mask_bad, "trim"] = (
        cars.loc[mask_bad, "trim"].fillna("") + 
        ", " + 
        cars.loc[mask_bad, "body"].fillna("")
    )
    # 2.3.2 restliche Spalten um eine Position zurückschieben
    cols = ["body","transmission","vin","state","condition","odometer",
            "color","interior","seller","mmr","sellingprice","saledate"]
    for i in range(len(cols)-1):
        cars.loc[mask_bad, cols[i]] = cars.loc[mask_bad, cols[i+1]]
    # 2.3.3 Neue Datumskonvertierung
    cars["saledate"] = pd.to_datetime(
        cars["saledate"], errors="coerce", utc=True
    ).dt.tz_convert(None)

# 2.4 Staaten-Tabelle einlesen
states = pd.read_excel(
    os.path.join(RAW_DIR, "US and Canda States.xlsx"),
    engine="openpyxl"
)

# 2.5 Überblick
print(f"cars:   {cars.shape[0]}×{cars.shape[1]}")
print(f"states: {states.shape[0]}×{states.shape[1]}")
display(cars.head(), states.head())


/tmp/ipykernel_44273/2570856548.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  cars = pd.read_csv(
/tmp/ipykernel_44273/2570856548.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cars = pd.read_csv(


,trim,body,saledate
408161,SE PZEV w/Connectivity,Navitgation,16500
417835,SE PZEV w/Connectivity,Navitgation,10500
421289,SE PZEV w/Connectivity,Navitgation,12700
424161,SE PZEV w/Connectivity,Navitgation,8250
427040,SE PZEV w/Connectivity,Navitgation,14300


/tmp/ipykernel_44273/2570856548.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['gray' 'silver' 'blue' 'white' 'gray' 'silver' 'silver' 'white' 'white'
 'black' 'silver' 'gray']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  cars.loc[mask_bad, cols[i]] = cars.loc[mask_bad, cols[i+1]]
/tmp/ipykernel_44273/2570856548.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan nan nan nan nan nan nan nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  cars.loc[mask_bad, cols[i]] = cars.loc[mask_bad, cols[i+1]]
/tmp/ipykernel_44273/2570856548.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specif

cars:   558837×16
states: 72×4


,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,kia motors america inc,20500.0,21500.0,2014-12-16 04:30:00
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,kia motors america inc,20800.0,21500.0,2014-12-16 04:30:00
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,45.0,1331.0,gray,black,financial services remarketing (lease),31900.0,30000.0,2015-01-14 20:30:00
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,41.0,14282.0,white,black,volvo na rep/world omni,27500.0,27750.0,2015-01-28 20:30:00
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,43.0,2641.0,gray,black,financial services remarketing (lease),66000.0,67000.0,2014-12-18 04:30:00


,StateCode,StateName,Region,AlternateName
0,AL,Alabama,Southeast US,Alabama
1,AK,Alaska,West US,Alaska
2,AS,American Samoa,US Territory,Samoa Americana
3,AZ,Arizona,Southwest US,Arizona
4,AR,Arkansas,Southeast US,Arkansas


## 3. Datentypen & Datumskomponenten
- Datentypen prüfen  
- Numerische Spalten casten  
- `saledate` → `datetime64`  
- `sale_year`,`sale_month`,`sale_day`,`sale_weekday` extrahieren

In [120]:
# 3.1 Datentypen vor Cast
print("Vor Cast:")
print(cars.dtypes, "\n")

# 3.2 Numerische Spalten casten
num_cols = ["condition","odometer","mmr","sellingprice"]
for c in num_cols:
    cars[c] = pd.to_numeric(cars[c], errors="coerce")

# 3.3 Datumskonvertierung (falls noch nicht typisiert)
cars["saledate"] = pd.to_datetime(
    cars["saledate"], errors="coerce", utc=True
).dt.tz_convert(None)

# 3.4 Datumskomponenten
cars["sale_year"]    = cars["saledate"].dt.year
cars["sale_month"]   = cars["saledate"].dt.month
cars["sale_day"]     = cars["saledate"].dt.day
cars["sale_weekday"] = cars["saledate"].dt.weekday  # 0=Mo … 6=So

# 3.5 Kontrolle
print("Nach Cast:")
print(cars[num_cols].dtypes)
print("NaT in saledate:", cars["saledate"].isna().sum(), "\n")
display(cars[["saledate","sale_year","sale_month","sale_day","sale_weekday"]].head())


Vor Cast:
year                     int64
make                    object
model                   object
trim                    object
body                    object
transmission            object
vin                     object
state                   object
condition              float64
odometer                object
color                   object
interior                object
seller                  object
mmr                    float64
sellingprice            object
saledate        datetime64[ns]
dtype: object 

Nach Cast:
condition       float64
odometer        float64
mmr             float64
sellingprice    float64
dtype: object
NaT in saledate: 38 



,saledate,sale_year,sale_month,sale_day,sale_weekday
0,2014-12-16 04:30:00,2014.0,12.0,16.0,1.0
1,2014-12-16 04:30:00,2014.0,12.0,16.0,1.0
2,2015-01-14 20:30:00,2015.0,1.0,14.0,2.0
3,2015-01-28 20:30:00,2015.0,1.0,28.0,2.0
4,2014-12-18 04:30:00,2014.0,12.0,18.0,3.0


## 4. State-Codes bereinigen & Merge
- `state` normalisieren  
- Ungültige Codes → `NaN`  
- Linker Merge mit `states` über `StateCode`  
- Fehlende Merges zählen  

In [121]:
# 4.1 state_clean
cars["state_clean"] = cars["state"].str.strip().str.upper()

# 4.2 Ungültige Längen und nicht in Liste → NaN
valid = set(states["StateCode"])
mask_len  = cars["state_clean"].str.len() != 2
mask_valid = ~cars["state_clean"].isin(valid)
cars.loc[mask_len | mask_valid, "state_clean"] = pd.NA

# 4.3 Merge
cars_states = cars.merge(
    states,
    how="left",
    left_on="state_clean",
    right_on="StateCode"
)

# 4.4 Kontrolle
n_unmatched = cars_states["StateCode"].isna().sum()
print(f"Nicht gematchte state_clean: {n_unmatched} von {len(cars_states)}")
display(cars_states.head())


Nicht gematchte state_clean: 38 von 558837


,year,make,model,trim,body,transmission,vin,state,condition,odometer,...,saledate,sale_year,sale_month,sale_day,sale_weekday,state_clean,StateCode,StateName,Region,AlternateName
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,...,2014-12-16 04:30:00,2014.0,12.0,16.0,1.0,CA,CA,California,West US,California
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,...,2014-12-16 04:30:00,2014.0,12.0,16.0,1.0,CA,CA,California,West US,California
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,45.0,1331.0,...,2015-01-14 20:30:00,2015.0,1.0,14.0,2.0,CA,CA,California,West US,California
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,41.0,14282.0,...,2015-01-28 20:30:00,2015.0,1.0,28.0,2.0,CA,CA,California,West US,California
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,43.0,2641.0,...,2014-12-18 04:30:00,2014.0,12.0,18.0,3.0,CA,CA,California,West US,California


## 5. Missing-Value-Report & Deskriptive Statistik
- `describe()` für Überblick  
- Fehlende Werte pro Spalte (absolut & Prozent)  


In [122]:
# 5.1 Deskriptive Statistik
display(cars_states.describe(include="all"))

# 5.2 Fehlende Werte
mv = cars_states.isna().sum().to_frame("n_missing")
mv["pct_missing"] = 100 * mv["n_missing"] / len(cars_states)
display(mv)

,year,make,model,trim,body,transmission,vin,state,condition,odometer,...,saledate,sale_year,sale_month,sale_day,sale_weekday,state_clean,StateCode,StateName,Region,AlternateName
count,558837.000000,548536,548438,548186,545641,493486,558833,558837,547017.000000,558731.000000,...,558799,558799.000000,558799.000000,558799.000000,558799.000000,558799,558799,558799,558799,558799
unique,NaN,96,973,1972,88,16,550289,71,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,38,38,38,7,38
top,NaN,Ford,Altima,Base,Sedan,automatic,automatic,fl,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,FL,FL,Florida,Southeast US,Florida
freq,NaN,93554,19349,55815,199429,475904,22,82945,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,82945,82945,82945,180779,82945
mean,2010.038927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.522523,68320.652287,...,2015-03-06 00:09:10.521385472,2014.903853,3.800801,14.634001,1.472229,NaN,NaN,NaN,NaN,NaN
min,1982.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,...,2014-01-01 01:15:00,2014.000000,1.000000,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN
25%,2007.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.000000,28370.500000,...,2015-01-20 18:00:00,2015.000000,1.000000,7.000000,1.000000,NaN,NaN,NaN,NaN,NaN
50%,2012.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.000000,52255.000000,...,2015-02-12 19:20:00,2015.000000,2.000000,16.000000,1.000000,NaN,NaN,NaN,NaN,NaN
75%,2013.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.000000,99110.500000,...,2015-05-21 19:00:00,2015.000000,6.000000,21.000000,2.000000,NaN,NaN,NaN,NaN,NaN
max,2015.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51648.000000,999999.000000,...,2015-07-20 19:30:00,2015.000000,12.000000,31.000000,6.000000,NaN,NaN,NaN,NaN,NaN


,n_missing,pct_missing
year,0,0.000000
make,10301,1.843292
model,10399,1.860829
trim,10651,1.905922
body,13196,2.361333
transmission,65351,11.694108
vin,4,0.000716
state,0,0.000000
condition,11820,2.115107
odometer,106,0.018968


## 6. Erste Datenbereinigung nach Merge
- Umgang mit NaNs in `saledate` und `state_clean`  
- Entfernen oder kennzeichnen der wenigen fehlerhaften Zeilen  

In [123]:
# 6.1 Zeilen ohne Datum entfernen (falls nicht benötigt)
before = len(cars_states)
cars_states = cars_states[cars_states["saledate"].notna()].reset_index(drop=True)
print(f"Entfernte Zeilen ohne Datum: {before - len(cars_states)}")

# 6.2 Zeilen ohne gültigen State markieren/entfernen
before = len(cars_states)
cars_states = cars_states[cars_states["state_clean"].notna()].reset_index(drop=True)
print(f"Entfernte Zeilen ohne gültigen State: {before - len(cars_states)}")

# 6.3 Aktueller Stand
print("Verbleibende Datensätze:", len(cars_states))


Entfernte Zeilen ohne Datum: 38
Entfernte Zeilen ohne gültigen State: 0
Verbleibende Datensätze: 558799


## 7. Erneute Missing-Value-Analyse


In [124]:
# Missing-Value-Report
missing_report = cars_states.isna().sum().to_frame(name="n_missing")
missing_report["pct_missing"] = 100 * missing_report["n_missing"] / len(cars_states)
display(missing_report.sort_values("pct_missing", ascending=False))

# Optional: nur Spalten mit fehlenden Werten
missing_report_nonzero = missing_report[missing_report["n_missing"] > 0]
print("\nSpalten mit NaN-Werten:")
display(missing_report_nonzero.sort_values("pct_missing", ascending=False))

,n_missing,pct_missing
transmission,65351,11.694903
body,13195,2.361314
condition,11794,2.110598
trim,10651,1.906052
model,10399,1.860955
make,10301,1.843418
color,749,0.134037
interior,749,0.134037
odometer,94,0.016822
state,0,0.000000



Spalten mit NaN-Werten:


,n_missing,pct_missing
transmission,65351,11.694903
body,13195,2.361314
condition,11794,2.110598
trim,10651,1.906052
model,10399,1.860955
make,10301,1.843418
color,749,0.134037
interior,749,0.134037
odometer,94,0.016822


## 6d. Entfernen und Imputation
- Entfernen aller Zeilen mit fehlenden Werten in transmission, body, trim oder make  
- Ausfüllen der fehlenden color-Werte mit der häufigsten Farbe

In [125]:
# 1. Zeilen löschen, in denen eine der kritischen Spalten fehlt
drop_cols = ["transmission", "body", "trim", "make"]
before_drop = len(cars_states)
cars_states = cars_states.dropna(subset=drop_cols).reset_index(drop=True)
print(f"Entfernte Zeilen: {before_drop - len(cars_states)}")

# 2. Häufigste Farbe ermitteln und fehlende Werte in color damit auffüllen
farbe_modus = cars_states["color"].mode().iloc[0]
print(f"Häufigste Farbe: {farbe_modus}")
cars_states["color"] = cars_states["color"].fillna(farbe_modus)

# 3. Kontrolle: Sind in transmission, body, trim, make oder color noch NaNs?
remaining_na = cars_states.isna().sum().loc[drop_cols + ["color"]]
print("\nVerbleibende NaN in transmission, body, trim, make, color:")
print(remaining_na)


Entfernte Zeilen: 76383
Häufigste Farbe: black

Verbleibende NaN in transmission, body, trim, make, color:
transmission    0
body            0
trim            0
make            0
color           0
dtype: int64


## 10. Zwischenergebnis speichern
- Alle Bereinigungen abgeschlossen  
- Export als CSV für das nächste Notebook  

In [126]:
# 7.1 Verbleibende DataFrame-Größe
print("Verbleibende Datensätze:", len(cars_states))

# 7.2 CSV speichern
out_path = os.path.join(PROCESSED_DIR, "cars_states_cleaned.csv")
cars_states.to_csv(out_path, index=False)
print("Zwischenergebnis gespeichert unter:", out_path)


Verbleibende Datensätze: 482416
Zwischenergebnis gespeichert unter: ../data/processed/cars_states_cleaned.csv
